In [2]:
from PIL import Image
import matplotlib.pyplot as plt

In [1]:
import os
import rasterio
import numpy as np

def check_band_coverage(directory):
    print(f"\n{'Band':<6} {'Resolution':<10} {'Shape':<15} {'Valid Pixels (%)':<18}")
    print("-" * 60)

    for fname in sorted(os.listdir(directory)):
        if not fname.endswith('.jp2') or '_TCI' in fname:
            continue

        band_code = fname.split('_')[-1].replace('.jp2', '')
        path = os.path.join(directory, fname)

        with rasterio.open(path) as src:
            img = src.read(1)
            total_pixels = img.size
            valid_pixels = np.count_nonzero(img)
            valid_pct = 100 * valid_pixels / total_pixels

            print(f"{band_code:<6} {src.res[0]:<10.1f} {img.shape!s:<15} {valid_pct:<18.2f}")


check_band_coverage("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34SEJ_20230118T233535.SAFE/GRANULE/L1C_T34SEJ_A032694_20210925T092343/IMG_DATA")
check_band_coverage("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34SFJ_20230118T233535.SAFE/GRANULE/L1C_T34SFJ_A032694_20210925T092343/IMG_DATA")
check_band_coverage("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34TEK_20230118T233535.SAFE/GRANULE/L1C_T34TEK_A032694_20210925T092343/IMG_DATA")
check_band_coverage("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34TFK_20230118T233535.SAFE/GRANULE/L1C_T34TFK_A032694_20210925T092343/IMG_DATA")


Band   Resolution Shape           Valid Pixels (%)  
------------------------------------------------------------
B01    60.0       (1830, 1830)    100.00            
B02    10.0       (10980, 10980)  100.00            


KeyboardInterrupt: 

In [1]:
import os
import rasterio
import numpy as np
import cv2

BANDS_10M = ['B02', 'B03', 'B04', 'B08']
BANDS_20M = ['B05', 'B06', 'B07', 'B8A', 'B11', 'B12']
BANDS_60M = ['B01', 'B09', 'B10']
ALL_BANDS = BANDS_10M + BANDS_20M + BANDS_60M

TARGET_SHAPE = (10980, 10980)

def pansharpen_to_10m(directory):
    band_paths = {}
    for fname in os.listdir(directory):
        if fname.endswith('.jp2') and '_TCI' not in fname:
            band_code = fname.split('_')[-1].replace('.jp2', '')
            if band_code in ALL_BANDS:
                band_paths[band_code] = os.path.join(directory, fname)

    missing = [b for b in ALL_BANDS if b not in band_paths]
    if missing:
        raise ValueError(f"Missing bands: {missing}")

    band_data = {}
    reference_meta = None

    for band in ALL_BANDS:
        with rasterio.open(band_paths[band]) as src:
            img = src.read(1)
            if band in BANDS_10M:
                band_data[band] = img
                if reference_meta is None:
                    reference_meta = src.meta.copy()
            else:
                resized = cv2.resize(img, (TARGET_SHAPE[1], TARGET_SHAPE[0]), interpolation=cv2.INTER_CUBIC)
                band_data[band] = resized

    ordered_bands = BANDS_60M + BANDS_10M + BANDS_20M
    stacked = np.stack([band_data[b] for b in ordered_bands], axis=0)

    reference_meta.update({
        "count": len(ordered_bands),
        "height": TARGET_SHAPE[0],
        "width": TARGET_SHAPE[1]
    })

    return stacked, reference_meta


In [ ]:
stacked_T34SEJ, reference_meta_T34SEJ = pansharpen_to_10m("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34SEJ_20230118T233535.SAFE/GRANULE/L1C_T34SEJ_A032694_20210925T092343/IMG_DATA")
stacked_T34SFJ, reference_meta_T34SFJ = pansharpen_to_10m("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34SFJ_20230118T233535.SAFE/GRANULE/L1C_T34SFJ_A032694_20210925T092343/IMG_DATA")
stacked_T34TEK, reference_meta_T34TEK = pansharpen_to_10m("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34TEK_20230118T233535.SAFE/GRANULE/L1C_T34TEK_A032694_20210925T092343/IMG_DATA")
stacked_T34TFK, reference_meta_T34TFK = pansharpen_to_10m("data/S2A_MSIL1C_20210925T092031_N0500_R093_T34TFK_20230118T233535.SAFE/GRANULE/L1C_T34TFK_A032694_20210925T092343/IMG_DATA")

print(stacked_T34SEJ.shape, stacked_T34SFJ.shape, stacked_T34TEK.shape, stacked_T34TFK.shape)

In [4]:
stacked.shape

(13, 10980, 10980)